# lab 4 - Wykorzystanie biblioteki Dask w zadaniach Machine Learning.

## 1. Eksperymetny bez wykorzystania biblioteki Dask.

In [ ]:
!pip install scikit-learn tqdm xgboost ipywidgets

**Przykład 1**

In [1]:
from sklearn.datasets import fetch_rcv1, fetch_covtype
import numpy as np

# pobranie zbioru RCV1 i zapisanie w postaci binarnej za pomocą ZARR
# dokumentacja sklearn z opcjami pobrania zbioru: https://scikit-learn.org/1.5/modules/generated/sklearn.datasets.fetch_rcv1.html#sklearn.datasets.fetch_rcv1
# rcv1 = fetch_rcv1()
# dane w oryginale są zwracane jako tablice rzadkie (sparse) i trzeba je zamienić
# na tablicę gęstą

# UWAGA: ta operacja spowoduje potrzebę zaalokowania około 283 GB pamięci RAM, co zapewne się nie uda
# więc tutaj trzeba będzie użyć klastra do realizacji tego zadania
# X, y = rcv1.data.toarray(), rcv1.target.toarray()

# mniejszy zbiór do przeprowadzenia przykładu
cov = fetch_covtype()
X, y = cov.data, cov.target


In [2]:
import dask.array as da

data = [X, da.atleast_2d(y).T]
ddf = da.concatenate(data, axis=1).to_dask_dataframe(columns=cov.feature_names + cov.target_names, index=None).compute()

In [3]:
ddf = ddf.reset_index()

In [8]:
ddf.head()

,index,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39,Cover_Type
0,0,2596.0,51.0,3.0,258.0,0.0,510.0,221.0,232.0,148.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
1,1,2590.0,56.0,2.0,212.0,-6.0,390.0,220.0,235.0,151.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
2,2,2804.0,139.0,9.0,268.0,65.0,3180.0,234.0,238.0,135.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,3,2785.0,155.0,18.0,242.0,118.0,3090.0,238.0,238.0,122.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
4,4,2595.0,45.0,2.0,153.0,-1.0,391.0,220.0,234.0,150.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0


In [9]:
ddf.describe()

,index,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39,Cover_Type
count,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,...,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000
mean,290505.500000,2959.365301,155.656807,14.103704,269.428217,46.418855,2350.146611,212.146049,223.318716,142.528263,...,0.090392,0.077716,0.002773,0.003255,0.000205,0.000513,0.026803,0.023762,0.015060,2.051471
std,167723.861639,279.984734,111.913721,7.488242,212.549356,58.295232,1559.254870,26.769889,19.768697,38.274529,...,0.286743,0.267725,0.052584,0.056957,0.014310,0.022641,0.161508,0.152307,0.121791,1.396504
min,0.000000,1859.000000,0.000000,0.000000,0.000000,-173.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,145252.750000,2809.000000,58.000000,9.000000,108.000000,7.000000,1106.000000,198.000000,213.000000,119.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,290505.500000,2996.000000,127.000000,13.000000,218.000000,30.000000,1997.000000,218.000000,226.000000,143.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
75%,435758.250000,3163.000000,260.000000,18.000000,384.000000,69.000000,3328.000000,231.000000,237.000000,168.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
max,581011.000000,3858.000000,360.000000,66.000000,1397.000000,601.000000,7117.000000,254.000000,254.000000,254.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.000000


In [10]:
# uwaga na faktyczny typ danych tej ramki!
type(ddf)

pandas.core.frame.DataFrame

In [5]:
import os
import fastparquet

DATADIR = './data/cov/'
os.makedirs(DATADIR, exist_ok=True)

In [12]:
# to pandas DataFrame (brak podziału na partycje)
ddf.to_parquet(os.path.join(DATADIR, 'data.parquet'))

# ddf.to_parquet(DATADIR, name_function=lambda x: f"data-{x}.parquet")

In [6]:
# jako, że ta wersja biblioteki XGBoost wymaga, aby wartości dla target(y) były indeksowane od 0
# należy wykonać enkodowanie dla tego datasetu

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)

In [ ]:
from sklearn.base import clone
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import StratifiedKFold, cross_validate
from tqdm.notebook import tqdm

import xgboost as xgb

# X, y = load_breast_cancer(return_X_y=True)
n_splits = 5

def fit_and_score(estimator, X_train, X_test, y_train, y_test):
    """Fit the estimator on the train set and score it on both sets"""
    estimator.fit(X_train, y_train, eval_set=[(X_test, y_test)])

    train_score = estimator.score(X_train, y_train)
    test_score = estimator.score(X_test, y_test)

    return estimator, train_score, test_score


cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=94)


# ta implementacja algorytmu XGBoost wykorzystuje joblib i domyślnie jego praca jest zrównoleglana
# wydajność użycia klasycznego podejścia vs. Dask na klastrze lokalnym prawdopodobnie przyniesie gorsze
# rezultaty dla tego drugiego rozwiązania, ale wymaga to sprawdzenia

# jednak jeżeli weźmiemy pod uwagę ograniczenia pamięci, które możemy napotkać pracując bez wykorzystania
# daska, może okazać się, że podejście klasyczne nie będzie możliwe do uruchomienia jeżeli nie
# dysponujemy wystarczającą ilością zasobów sprzętowych

clf = xgb.XGBClassifier(tree_method="hist", early_stopping_rounds=3)

results = {}

for train, test in tqdm(cv.split(X, y), desc="Training...", total=n_splits):
    X_train = X[train]
    X_test = X[test]
    y_train = y[train]
    y_test = y[test]
    est, train_score, test_score = fit_and_score(
        clone(clf), X_train, X_test, y_train, y_test
    )
    results[est] = (train_score, test_score)

Training...:   0%|          | 0/5 [00:00<?, ?it/s]

[0]	validation_0-mlogloss:1.42195
[1]	validation_0-mlogloss:1.17182
[2]	validation_0-mlogloss:1.00835
[3]	validation_0-mlogloss:0.89389
[4]	validation_0-mlogloss:0.81015
[5]	validation_0-mlogloss:0.74716
[6]	validation_0-mlogloss:0.69977
[7]	validation_0-mlogloss:0.66108
[8]	validation_0-mlogloss:0.63163
[9]	validation_0-mlogloss:0.60702
[10]	validation_0-mlogloss:0.58751
[11]	validation_0-mlogloss:0.56978
[12]	validation_0-mlogloss:0.55626
[13]	validation_0-mlogloss:0.54217
[14]	validation_0-mlogloss:0.52944
[15]	validation_0-mlogloss:0.52098
[16]	validation_0-mlogloss:0.51338
[17]	validation_0-mlogloss:0.50204
[18]	validation_0-mlogloss:0.49490
[19]	validation_0-mlogloss:0.48963
[20]	validation_0-mlogloss:0.48454
[21]	validation_0-mlogloss:0.48032
[22]	validation_0-mlogloss:0.47495
[23]	validation_0-mlogloss:0.47029
[24]	validation_0-mlogloss:0.46705
[25]	validation_0-mlogloss:0.46266
[26]	validation_0-mlogloss:0.45989
[27]	validation_0-mlogloss:0.45814
[28]	validation_0-mlogloss:0.4

In [ ]:
for fold_n, fold in enumerate(results.items(), start=1):
    print(f"Fold {fold_n}: train_score: {fold[1][0]}, test_score: {fold[1][1]}")

## 2. Eksperyment z wykorzystaniem Dask.

### 2.1 

**Przykład 2**

In [ ]:
import dask
import dask.dataframe as dd
import dask.array as da
from dask_ml.model_selection import KFold
import xgboost as xgb
from dask.distributed import LocalCluster

In [ ]:
# konwersja numpy array na Dask array
X = da.from_array(X)
y = da.from_array(y)

In [ ]:
# szczegóły definiowania parametrów: https://xgboost.readthedocs.io/en/latest/parameter.html#learning-task-parameters
# tu klasyfikacja wieloklasowa

params = {'objective': 'multi:softmax',
          'max_depth': 4, 'eta': 0.01, 'subsample': 0.5,
          'min_child_weight': 0.5,
          'num_class': 7}


n_splits = 5
cv = KFold(n_splits=n_splits, shuffle=True, random_state=94)

predictions = {}

with LocalCluster(n_workers=4) as cluster:
    display(cluster)
    with cluster.get_client() as client:

        for i, (train, test) in enumerate(cv.split(X, y)):
            
            X_train = X[train, :-1]
            X_test = X[test, :-1]
            y_train = y[train]
            y_test = y[test]
            
            d_train = xgb.dask.DaskDMatrix(client, X_train, y_train, enable_categorical=True)
            model = xgb.dask.train(client, params=params, dtrain=d_train)
            predictions[f'fold_{i}'] = xgb.dask.predict(client, model, X_test)


### 2.2 Rozbicie całego procesu na niezależne zadania

W pierwszej fazie zazwyczaj zajęlibyśmy się procesem ekstrakcji, oczyszczania i wstępnego przetwarzania danych, jednak póki co ten etap zostanie tutaj pominięty.

W dokumentacji biblioteki Dask znajdziemy cały dział poruszający temat Machine Learning z wykorzystaniem tej biblioteki na różnych etapach tego procesu.

> Dokumentacja Dask ML: https://ml.dask.org/

Są tam również przykłady (chociaż dość ubogie i czasem nie działające z najnowszymi wersjami bibliotek zależnych), które obrazują w jakich przypadkach można skorzystać z możliwości Dask w kontekście ML. Należy dość dokładnie przeczytać uwagi i wskazówki, które się tam znajdują gdyż nie wszystkie elementy (np. znane biblioteki do tworzenia modeli ML) współpracują z Dask w tzw. trybie "out of the box" i trzeba wykorzystywać wrappery lub specjalnie przygotowane integracje, np. bibliotekę Skorch, która pozwala korzystać z Pytorch w sposób bardziej kompatybilny ze scikit-learn a co za tym idzie również z Dask, gdyż tutaj położono największy nacisk na integrację.

In [ ]:
# 1. ładowanie danych
# w zależności od potrzeb dane mogą być przekazane do etapu treningowego w różnym formacie: dask dataframe, dask array lub inny.
# dodatkowo dzięki wielu formatom przechowywania danych, szczególnie w kontekście big data, gdzie właściwy dobór bibliotek zależy od
# dostępnej infrastruktury.
# Dane mogą również być wczytywane i następnie przetwarzane w paczkach (ang. batch), co dodatkowo może narzucić pewne ograniczenia co
# do miejsca uruchomienia procesu ładowania danych (może tylko scheduler, a może zdalnie na klastrze) lub konieczność rozproszenia
# danych po całym klastrze.




In [ ]:
# 2. Podział danych na potrzeby etapu trenowania modelu.
# Jeżeli docelowo dysponujemy dużym zbiorem danych, nie oznacza to wcale, że tak jak klasycznie zazwyczaj się to odbywa,
# dzielimy cały zbiór na część treningową oraz testową (np. w proporcjach 80/20, 70/30 czy innych) i uruchamiamy na nich
# trening wybranego modelu. Lepszym pomysłem jest dobranie odpowiedniej, reprezentatywnej próbki danych, na których wykonamy
# wstępny trening. W zależności od różnorodności zbioru może się okazać, że wyniki będzie wystarczająco dobry. Pamiętajmy również, że
# finalna wielkość modelu i ilość zasobów potrzebnych, żeby go przechowywać w stanie dostępnym dla etapu inferencji może być kosztowne
# i również często wymaga pewnej optymalizacji. A skoro w danym przypadku mniejszy model jest porównywalnie dobry z modelem większym,
# mniejszy wygrywa. Również w kontekście szybkości inferencji w fazie produkcyjnej.


# zobacz przykład podziału danych w punkcie 2.1

In [ ]:
# 3. Trening modelu.
# Ta faza zazwyczaj zajmuje dużo czasu, w kontekście samego treningu, ale również w kontekście iteracyjnej natury tego etapu.
# Szukamy optymalnych parametrów modelu (tu mogą pomóc dodatkowe narzędzia i techniki takie jak Optune, ML Flow, grid search),
# eksperymentujemy z doborem danych (tu często poprzedzamy to fazą feature engineering).

# trening modelu na dużej ilości danych można wykonać na kilka sposobów.
# 1. Użycie modelu ensemble, który trenuje większą ilość mniejszych modeli na fragmentach danych.
# Ważne jest, aby podział danych odbył się zgodnie z rozkładem w zbiorze oryginalnym, w przeciwnym wypadku część
# modeli może dość mocno wpływać na ogólny wynik całego modelu.
# zobacz: https://ml.dask.org/modules/generated/dask_ml.ensemble.BlockwiseVotingClassifier.html#dask_ml.ensemble.BlockwiseVotingClassifier

# 2. Wykorzystanie jednej z klas biblioteki Dask, która pozwala wykorzystać modele z biblioteki scikit-learn, które wspierają
# operację partial_fit, która pozwala na trenowanie modelu na zbiorze danych uczących, który jest podzielony na części (nie 
# mylić z paczką, ang. batch, która jest dzielona ze zbioru treningowego) i dzięki temu można tu przekazać np. Dask Array jako dane wejściowe.
# zobacz: https://ml.dask.org/modules/generated/dask_ml.wrappers.Incremental.html#dask_ml.wrappers.Incremental
# oraz https://scikit-learn.org/0.15/modules/scaling_strategies.html

# przykłady

# przykład z ofocjalnej dokumentacji: https://examples.dask.org/machine-learning/incremental.html
# inne przykłady
# https://skorch.readthedocs.io/en/stable/user/parallelism.html
# https://github.com/skorch-dev/skorch/blob/master/notebooks/MNIST.ipynb

**Przykład 3**

In [ ]:
from dask.distributed import Client

# pamiętaj o zamykaniu klientów lub używaniu już wcześniej stworzonego
client = Client(n_workers=4, threads_per_worker=2, memory_limit="4GB")
client

In [ ]:
# przykład z użyciem incremental learning z dokumentacji dask
# z lekką modyfikacją

import dask
import dask.array as da
from dask_ml.datasets import make_classification
from dask_ml.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from dask_ml.wrappers import Incremental


# dostosuj wielkość zbioru oraz ilość/wielkość chunka
# wielkość chunka niedobrana prawidłowo do pamięci na workerze może
# skutecznie zakończyć przeliczanie całego grafu
# przy parametrach poniżej potrzeba około 38GB pamięci RAM, ale przy dobrze
# dobranych chunkach obliczymy to na dużo mniejszej ilości zasobów
# wykonanie poniższego kodu na mojej maszynie z zadanymi parametrami klastra lokalnego
# zajęło kilkanaście minut
n, d = 10000000, 500
X, y = make_classification(n_samples=n, n_features=d,
                           chunks=n // 64, flip_y=0.2)
display(X)

X_train, X_test, y_train, y_test = train_test_split(X, y)
display(X_train)

# jeżeli dysponujemy wystarczająco dużą ilością pamięci RAM rozproszoną po workerach
# to możemy przechować dane właśnie tam w celu przyspieszenia części obliczeń
# X_train, X_test, y_train, y_test = dask.persist(X_train, X_test, y_train, y_test)


classes = da.unique(y_train).compute()
# classes

est = SGDClassifier(loss='log_loss', penalty='l2', tol=1e-3)
inc = Incremental(est, scoring='accuracy')

inc.fit(X_train, y_train, classes=classes)
inc.score(X_test, y_test)

In [ ]:
# 4. Serializacja modelu i jego wczytywanie.
# Serializacja modelu jest konieczna ze względu na chęć przechowania go w formie bardziej trwałej niż w pamięci operacyjnej, ale
# również ze względu na niski koszt jego wczytania wględem konieczności ponownego jego trenowania, to oczywiste.
# Często również w cyklu życia modeli ML następuje ich aktualizacja oraz archiwizacja modeli aktualnie nie używanych.


def save_model(model):
    pass

def load_model(path):
    pass


**Przykład 4**

In [ ]:
# 5. Inferencja
# W tej fazie podajemy do modelu dane, w kontekście których cały ten proces był wykonany. Chcemy się dowiedzieć kto, z jaką
# szansą porzuci w niedalekiej przyszłości naszą usługę, co jest na zdjęciu lub czy na zdjęciu jest coś co nas szczególnie interesuje,
# a może spełniamy prośbę użytkownika o wygenerowanie zabawnego tekstu życzeń urodzinowych dla najlepszego kolegi.

# przykładowy przepływ z dokumentacji dask - batch prediction
# przykład nie jest kompletny


from dask.distributed import LocalCluster

cluster = LocalCluster(processes=False)
client = cluster.get_client()

# tu możemy wykorzystać poznany już Dask Bag
filenames = [...]

def predict(filename, model):
    data = load(filename)
    result = model.predict(data)
    return result

model = client.submit(load_model, path_to_model)
predictions = client.map(predict, filenames, model=model)
# czekamy na wszystkie wyniki
results = client.gather(predictions)

# lub wykorzystując przykład z lab_3 z użyciem dask.distributed.as_completed możemy odbierać wyniki paczkami i przetwarzać je dalej

**Przykład 5**

In [ ]:
# skrypt pokazujący jak można wykorzystać Dask do rozproszonego
# poszukiwania najbardziej optymalnych hiperparametrów danego klasyfikatora z wybranymi danymi
# Dzięki temu możemy na niewielkiej próbce danych (ale reprezentatywnej) dobrać
# hiperparametry modelu i przejść do szkolenia modelu docelowego na większej ilości danych

from dask_ml.model_selection import IncrementalSearchCV
import numpy as np
from dask_ml.datasets import make_classification
from sklearn.linear_model import SGDClassifier


X, y = make_classification(n_samples=5000000, n_features=20,
                           chunks=100000, random_state=56)


model = SGDClassifier(tol=1e-3, penalty='elasticnet', random_state=0)

params = {'alpha': np.logspace(-2, 1, num=1000),
          'l1_ratio': np.linspace(0, 1, num=1000),
          'average': [True, False]}

# search = IncrementalSearchCV(model, params, random_state=0)

search = IncrementalSearchCV(model, params, random_state=0,
                             n_initial_parameters=1000,
                             patience=20, max_iter=100)
search.fit(X, y, classes=[0, 1])

In [ ]:
# najlepszy model oraz najlepsze parametry
# więcej o tym przykładzie: 
# https://ml.dask.org/modules/generated/dask_ml.model_selection.IncrementalSearchCV.html#dask_ml.model_selection.IncrementalSearchCV

search.best_score_, search.best_params_

In [ ]:
client.close()

### Zadania

**Zadanie 1**  
Uruchom przykład Incremental learning z punktu 2.2 (przykład 3) dobierając parametry tak, aby ilość danych do przeliczenia była większa niż sumaryczna ilość pamięci RAM workerów. Obserwuj dashboard i w razie niepowodzenia dostosuj wielkość i ilość chunków tak, aby obliczenia się wykonały na tych samych parametrach workerów. Zobacz jak wygląda struktura pamięci na workerach, czy nie dochodzi do zrzucania pamięci na dysk (zapewne będzie on wąskim gardłem, więc w menedżerze będzie widać jego mocne obciążenie). Zastanów się czy można to jakoś zoptymalizować przy dostępnych workerach i wykonaj kilka eksperymentów szukając większej wydajności i krótszego czasu wykonania całego zadania.

**Zadanie 2**  
Dokonaj serializacji modelu z zadania 1 na dysk i następnie go wczytaj ponownie tak, aby można było uruchomić na nim predykcję dla tablic X_test oraz y_test (dla użycia miar klasyfikacji) i wyświetl macierz klasyfikacji (confusion matrix).

**Zadanie 3**  
Korzystając z danych stworzonych w zadaniu 1 uruchom poszukiwanie optymalnych parametrów modelu tak jak zostało to zaprezentowane w przykładzie 5. Ta metoda powinna sama wybierać modele obiecujące i trenować je na większej liczbie danych porzucając jednocześnie modele, które nie rokują.
Sprawdź jak wyglądają najlepsze wyliczone parametry vs. te użyte w zadaniu 1 i ewentualnie dopasuj próbkę danych jeżeli jej inicjalna wielkość nie pozwala na wykonanie zadania (zwróć uwagę na ilość i wielkość chunków w przykładzie 3 oraz 5, w tym drugim jest ich znacznie więcej, co przyspiesza poszukiwanie optymalnych parametrów).